In [22]:
import numpy as np
import matplotlib.pyplot as plt

In [23]:
import pickle

In [115]:
params = open('/home/bilal/workspace/UmeshSejwani/variable_autoencoder/experiments/stickBreaking_vae_params_.pkl', 'rb')

In [116]:
data = pickle.load(params)

In [117]:
import numpy as np
import theano
import theano.tensor as T

def Sigmoid(x):
    y = T.nnet.sigmoid(x)
    return(y)

def ReLU(x):
    y = T.maximum(0.0, x)
    return(y)

def Softplus(x):
    y = T.nnet.softplus(x)
    return(y)

class HiddenLayer(object):
    def __init__(self, rng, input, n_in, n_out, activation, W=None, b=None):
        self.input = input
        self.activation = activation

        if W is None:
            W_values = np.asarray(0.01 * rng.standard_normal(size=(n_in, n_out)), dtype=theano.config.floatX)
            W = theano.shared(value=W_values, name='W')
        if b is None:
            b_values = np.zeros((n_out,), dtype=theano.config.floatX)
            b = theano.shared(value=b_values, name='b')
        self.W = W
        self.b = b

        self.output = self.activation( T.dot(self.input, self.W) + self.b )
    
        # parameters of the model
        self.params = [self.W, self.b]

class Decoder(object):
    def __init__(self, rng, input, latent_size, out_size, activation, W_z = None, b = None):
        self.input = input
        self.activation = activation

        # setup the params                                                                                                                          
        if W_z is None:
            W_values = np.asarray(0.01 * rng.standard_normal(size=(latent_size, out_size)), dtype=theano.config.floatX)
            W_z = theano.shared(value=W_values, name='W_hid_z')
        if b is None:
            b_values = np.zeros((out_size,), dtype=theano.config.floatX)
            b = theano.shared(value=b_values, name='b')
        self.W_z = W_z
        self.b = b
        
        self.pre_act_out = T.dot(self.input, self.W_z) + self.b
        self.output = self.activation(self.pre_act_out)
        
        # gather parameters
        self.params = [self.W_z, self.b]
        
class StickBreakingEncoder(object):
    def __init__(self, rng, input, batch_size, in_size, latent_size, W_a = None, W_b = None, epsilon = 0.01):
        self.srng = theano.tensor.shared_randomstreams.RandomStreams(rng.randint(999999))
        self.input = input

        # setup variational params
        if W_a is None:
            W_values = np.asarray(0.01 * rng.standard_normal(size=(in_size, latent_size-1)), dtype=theano.config.floatX)
            W_a = theano.shared(value=W_values, name='W_a')
        if W_b is None:
            W_values = np.asarray(0.01 * rng.standard_normal(size=(in_size, latent_size-1)), dtype=theano.config.floatX)
            W_b = theano.shared(value=W_values, name='W_b')
        self.W_a = W_a
        self.W_b = W_b

        # compute Kumaraswamy samples
        uniform_samples = T.cast(self.srng.uniform(size=(batch_size, latent_size-1), low=0.01, high=0.99), theano.config.floatX)
        self.a = Softplus(T.dot(self.input, self.W_a))
        self.b = Softplus(T.dot(self.input, self.W_b))
        v_samples = (1-(uniform_samples**(1/self.b)))**(1/self.a)

        # setup variables for recursion
        stick_segment = theano.shared(value=np.zeros((batch_size,), dtype=theano.config.floatX), name='stick_segment')
        remaining_stick = theano.shared(value=np.ones((batch_size,), dtype=theano.config.floatX), name='remaining_stick')

        def compute_latent_vars(i, stick_segment, remaining_stick, v_samples):
            # compute stick segment
            stick_segment = v_samples[:,i] * remaining_stick
            remaining_stick *= (1-v_samples[:,i])
            return (stick_segment, remaining_stick)

        (stick_segments, remaining_sticks), updates = theano.scan(fn=compute_latent_vars,
                                                                  outputs_info=[stick_segment, remaining_stick],sequences=T.arange(latent_size-1),
                                                                  non_sequences=[v_samples], strict=True)

        self.avg_used_dims = T.mean(T.sum(remaining_sticks > epsilon, axis=0))
        self.latent_vars = T.transpose(T.concatenate([stick_segments, T.shape_padaxis(remaining_sticks[-1, :],axis=1).T], axis=0))

        self.params = [self.W_a, self.W_b]

    # Kumaraswamy distribution
    def calc_kl_divergence(self, prior_alpha, prior_beta):
        # compute taylor expansion for E[log (1-v)] term
        # hard-code so we don't have to use Scan()
        kl = 1./(1+self.a*self.b) * Beta_fn(1./self.a, self.b)
        kl += 1./(2+self.a*self.b) * Beta_fn(2./self.a, self.b)
        kl += 1./(3+self.a*self.b) * Beta_fn(3./self.a, self.b)
        kl += 1./(4+self.a*self.b) * Beta_fn(4./self.a, self.b)
        kl += 1./(5+self.a*self.b) * Beta_fn(5./self.a, self.b)
        kl += 1./(6+self.a*self.b) * Beta_fn(6./self.a, self.b)
        kl += 1./(7+self.a*self.b) * Beta_fn(7./self.a, self.b)
        kl += 1./(8+self.a*self.b) * Beta_fn(8./self.a, self.b)
        kl += 1./(9+self.a*self.b) * Beta_fn(9./self.a, self.b)
        kl += 1./(10+self.a*self.b) * Beta_fn(10./self.a, self.b)
        kl *= (prior_beta-1)*self.b

        # use another taylor approx for Digamma function
        psi_b_taylor_approx = T.log(self.b) - 1./(2 * self.b) - 1./(12 * self.b**2)
        kl += (self.a-prior_alpha)/self.a * (-0.57721 - psi_b_taylor_approx - 1/self.b) #T.psi(self.posterior_b)

        # add normalization constants
        kl += T.log(self.a*self.b) + T.log(Beta_fn(prior_alpha, prior_beta))

        # final term
        kl += -(self.b-1)/self.b

        return kl.sum(axis=1)

In [118]:
rng = np.random.RandomState(1234)

In [119]:
x = T.matrix('x')

In [120]:
data[0].shape, data[1].shape

((784, 500), (500,))

In [121]:
hidden = HiddenLayer(rng, x, 784, 500, ReLU, W=data[0], b=data[1])

In [122]:
encoder = StickBreakingEncoder(rng, x, 1, 500, 50, W_a=data[2], W_b=data[3])

In [123]:
hidden_run = theano.function(
    inputs = [hidden.input],
    outputs = hidden.output
)

In [124]:
encoder_run = theano.function(
    inputs = [encoder.input],
    outputs = encoder.latent_vars
)

In [125]:
import gzip
import numpy as np
import cPickle

In [126]:
with gzip.open('/home/bilal/workspace/UmeshSejwani/variable_autoencoder/datasets/mnist.pkl.gz', 'rb') as f:
    trainset, validset, testset = cPickle.load(f)

In [127]:
encoded_train_set = np.zeros((trainset[0].shape[0], 50))

In [128]:
for i in range(trainset[0].shape[0]):
    out_hidden = hidden_run(trainset[0][i].reshape(1, 784))
    out_encoded = encoder_run(out_hidden)
    encoded_train_set[i] = out_encoded

In [129]:
encoded_test_set = np.zeros((testset[0].shape[0], 50))

In [130]:
for i in range(testset[0].shape[0]):
    out_hidden = hidden_run(trainset[0][i].reshape(1, 784))
    out_encoded = encoder_run(out_hidden)
    encoded_test_set[i] = out_encoded

In [131]:
from sklearn.neighbors import KNeighborsClassifier

In [132]:
neigh = KNeighborsClassifier(n_neighbors=3)

In [133]:
neigh.fit(encoded_train_set, trainset[1])

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=3, p=2,
           weights='uniform')

In [134]:
neigh.score(encoded_test_set, testset[1])*100

9.950000000000001

In [135]:
neigh = KNeighborsClassifier(n_neighbors=5)

In [136]:
neigh.fit(encoded_train_set, trainset[1])

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')

In [137]:
neigh.score(encoded_test_set, testset[1])*100

9.879999999999999

In [138]:
neigh = KNeighborsClassifier(n_neighbors=10)

In [139]:
neigh.fit(encoded_train_set, trainset[1])

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=10, p=2,
           weights='uniform')

In [140]:
neigh.score(encoded_test_set, testset[1])*100

9.85